In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
import os

#데이터 불러오기
path='/content/drive/MyDrive/2021-1학기/파데분/데이터/'
elevator= pd.read_csv(path+"서울교통공사 교통약자 이용시설(승강기) 가동현황.csv", encoding='cp949')
restroom= pd.read_csv(path+"서울교통공사_역사공중화장실정보_20200928.csv",encoding='cp949')
amenities= pd.read_csv(path+"서울교통공사_편의시설 현황_20200917.csv", encoding='cp949')

# 데이터 살펴보기

# 1) 승강기 가동현황

In [23]:
#역코드 순서(150번 부터~)로 정렬되어 있음. 
elevator.head(5)

,역코드,역명,승강기명,운행구간,설치위치,운행상태,승강기 구분
0,150,서울(1)역,에스컬레이터 1호기,~,NaN,사용가능,ES
1,150,서울(1)역,에스컬레이터 1호기,B2(대합실)~B1(연결통로),4호선 연결통로 상행,사용가능,ES
2,150,서울(1)역,에스컬레이터 2호기,섬식 B1(연결통로)~B2(승강장),남영방향 하행 2-3,사용가능,ES
3,150,서울(1)역,에스컬레이터 3호기,섬식 B2(승강장)~B1(연결통로),시청방향 상행 9-3,사용가능,ES
4,150,서울(1)역,에스컬레이터 4호기,지상~B1,4번출구하행,사용가능,ES


In [4]:
elevator.tail(5)

,역코드,역명,승강기명,운행구간,설치위치,운행상태,승강기 구분
3009,2827,모란(8),MRL엘리베이터 3호기,F1-B1,11번출구,사용가능,EV
3010,2827,모란(8),에스컬레이터 1호기,B1-B2,지하 B1-B2,사용가능,ES
3011,2827,모란(8),에스컬레이터 2호기,B1-B2,지하 B1-B2,사용가능,ES
3012,2827,모란(8),휠체어리프트 1호기,환승수진측,지하 환승수진측,사용가능,WL
3013,2827,모란(8),휠체어리프트 2호기,환승복정측,지하 환승복정측,사용가능,WL


In [13]:
# '운행구간' 2618개, '설치위치' 2959개로 총 3014개 중 각각 빈 값이 있음.
# 왜 빈 값이 존재? -> 1)전 구간을 운행해서 2)설치가 되어있지 않아서
# 만약 가장 가까운 승강기를 찾아주는 프로그램이라면, 빈 값을 어떻게 할 것인지도 논의

elevator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3014 entries, 0 to 3013
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역코드     3014 non-null   int64 
 1   역명      3014 non-null   object
 2   승강기명    3014 non-null   object
 3   운행구간    2618 non-null   object
 4   설치위치    2959 non-null   object
 5   운행상태    3014 non-null   object
 6   승강기 구분  3014 non-null   object
dtypes: int64(1), object(6)
memory usage: 165.0+ KB


역코드

-> 역사공중화장실 데이터에는 역코드번호가 없음
(편의시설 데이터에는 역코드번호 있음) 

In [60]:
elevator_code_list = elevator['역코드'].unique()
elevator_code_list

array([ 150,  151,  152,  153,  154,  155,  156,  157,  158,  159,  201,
        202,  203,  204,  205,  206,  207,  208,  209,  210,  211,  212,
        213,  214,  215,  216,  217,  218,  219,  220,  221,  222,  223,
        224,  225,  226,  227,  228,  229,  230,  231,  232,  233,  234,
        235,  236,  237,  238,  239,  240,  241,  242,  243,  244,  245,
        246,  247,  248,  249,  250,  309,  310,  311,  312,  313,  314,
        315,  316,  317,  318,  319,  320,  322,  323,  324,  325,  326,
        327,  328,  329,  330,  331,  332,  333,  334,  335,  336,  337,
        338,  339,  340,  341,  342,  409,  410,  411,  412,  413,  414,
        415,  416,  417,  418,  419,  420,  421,  422,  423,  424,  425,
        426,  427,  428,  429,  430,  431,  432,  433,  434, 2511, 2512,
       2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523,
       2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534,
       2535, 2536, 2537, 2539, 2540, 2541, 2542, 25

In [62]:
#코드는 총 281개
#왜 역 코드(277개)와 역명 총 숫자가 다르지? -> 환승역 중복 때문. 가령 시청역(1)은 132, 시청역(2)는 201이다.
print(len(elevator_code_list))
print(elevator_code_list[280]) #마지막 코드 번호

281
2827


: 역 코드와 역명의 총 숫자가 차이가 나는 이유?

*의문점) 왜 281-277 = 4 개밖에 차이가 안나지? 환승역은 4개보다 많음

**승강기 데이터의 환승역 분류**

1) 각자 다른 역코드로 중복 계산되는 환승역 정보

e.g. 가락시장역, 건대입구(212, 2729), 교대(223, 330),  군자(2545, 2727), 노원(411,2715) ...

2) 환승역임에도 중복값이 아닌 경우

e.g. 가산디지털단지역(2748)...

3) 1-8호선 안에 포함되지 않는 환승역 -> 이 경우는 제외시키면 되니 고려대상은 아닌 듯

e.g. 강남역...


(참고) 환승역 정보
https://namu.wiki/w/%ED%99%98%EC%8A%B9%EC%97%AD

https://m.blog.naver.com/PostView.nhn?blogId=igl119&logNo=220937223383&proxyReferer=https:%2F%2Fwww.google.com%2F

**참고 <지하철 역 개수> 2017.09월 기준**

https://news.seoul.go.kr/traffic/archives/1551

1호선 10

2호선 순환 43

2호선 지선 4

2호선 지선 3

3호선 34

4호선 26

5호선 51

6호선 38

7호선 51

8호선 17

9호선 30

우이 신설선 13

=> 승강기 가동현황 데이터는 1호선~8호선까지밖에 없음!
(1호선부터 8호선까지의 역만 더하면 277개가 된다.)

이유는 모르겠으나, 9호선의 승강기 개수는 서울시메트로9호선(주), 서울메트로9호선운영(주)로 따로 개수를 세고 있다. 자세한 내용은 해당 url 참고.

역명

In [63]:
# .unique()를 사용하여 역명에 어떤 값이 있는지 살펴보기
#역명에 (1) 등 호선을 의미하는 숫자가 붙어있음. 합치게 된다면 고려해야함.
elevator_list = elevator['역명'].unique()
elevator_list

array(['서울(1)역', '시청(1)', '종각', '종로3가(1)', '종로5가', '동대문(1)', '신설동(1)',
       '제기동', '청량리', '동묘앞', '시청(2)', '을지로입구', '을지로3가(2)', '을지로4가',
       '동대문역사문화공원(2)', '신당', '상왕십리', '왕십리', '한양대', '뚝섬', '성수', '건대입구',
       '구의', '강변', '잠실나루', '잠실', '잠실새내', '종합운동장', '삼성', '선릉', '역삼', '강남',
       '교대(2)', '서초', '방배', '사당(2)', '낙성대', '서울대입구', '봉천', '신림', '신대방',
       '구로디지털단지', '대림', '신도림', '문래', '영등포구청', '당산', '합정', '홍대입구', '신촌',
       '이대', '아현', '충정로', '용답', '신답', '신설동(2)', '도림천', '양천구청', '신정네거리',
       '용두', '지축', '구파발', '연신내', '불광', '녹번', '홍제', '무악재', '독립문', '경복궁',
       '안국', '종로3가(3)', '을지로3가(3)', '동대입구', '약수', '금호', '옥수', '압구정', '신사',
       '잠원', '고속터미널', '교대(3)', '남부터미널', '양재', '매봉', '도곡', '대치', '학여울',
       '대청', '일원', '수서', '가락시장', '경찰병원', '오금', '당고개', '상계', '노원', '창동',
       '쌍문', '수유(강북구청)', '미아', '미아사거리', '길음', '성신여대입구', '한성대입구', '혜화',
       '동대문(4)', '동대문역사문화공원(4)', '충무로(4)', '명동', '회현', '서울(4)역', '숙대입구',
       '삼각지', '신용산', '이촌', '동작', '총신대입구', '사당(4)', '남태령', '방화', '개화

In [41]:
#elevator의 역명은 총 277개(0번 ~ 276번)
len(elevator_list)

277

In [39]:
print(elevator_list[276])   #마지막 역

모란(8)


In [67]:
elevator['역명'].value_counts()

고속터미널         54
가락시장          43
동묘앞           36
하남시청(덕풍신장)    32
오금            32
              ..
용답             2
을지로3가(2)       2
동작             2
신답             2
지축             1
Name: 역명, Length: 277, dtype: int64

In [20]:
#elevator['역명'].value_counts().plot(kind="barh")

승강기명

In [75]:
#에스컬레이터, 엘리베이터, 휠체어 리프트, 내부/외부의 여부 파악 가능
#승강기명으로 승강기를 찾지는 않으나, 승강기의 유형정도는 파악하게 하는 것이 좋을 듯 -> 이라고 생각했으나 승강기 구분이 따로 있어 딱히 필요하지는 않은 듯함
elevator['승강기명'].unique()

array(['에스컬레이터 1호기', '에스컬레이터 2호기', '에스컬레이터 3호기', '에스컬레이터 4호기',
       '에스컬레이터 5호기', '엘리베이터 내부 1호', '엘리베이터 외부 1호기', '엘리베이터 외부 2호기',
       '엘리베이터 외부3호기', '휠체어리프트 내부 1호기', '엘리베이터 내부 1호기', '엘리베이터 내부 2호기',
       '에스컬레이터 2호기(상행)', '엘리베이터 외부1호기', '엘리베이터 내부 2호', '엘리베이터 내부1호기',
       '엘리베이터 내부2호기', '엘리베이터 외부 3호기', '휠체어리프트 내부 2호기', '휠체어리프트 내부 3호기',
       '휠체어리프트 내부 4호기', '휠체어리프트 외부 1호기', '휠체어리프트 외부 2호기', '에스컬레이터 10호기',
       '에스컬레이터 11호기', '에스컬레이터 12호기', '에스컬레이터 6호기', '에스컬레이터 7호기',
       '에스컬레이터 8호기', '에스컬레이터 9호기', '엘리베이터 내부3호기', '엘리베이터 내부4호기',
       '엘리베이터 내부5호기', '엘리베이터 외부2호기', '에스컬레이터 1호기-휴지중', '에스컬레이터 2호기-휴지중',
       '휠체어리프트 외부1호기-휴지중(철거예정)', '휠체어리프트 내부 1호', '엘리베이터 외부 4호기',
       '휠체어리프트 내부1호기', '에스켈레이터 1호기', '에스켈레이터 2호기', '에스켈레이터 3호기',
       '엘리베이터-문래역 1번출구측 엘리베이터 외부2호기', '에스컬레이터-이대역 E/S 1호기 [192070]',
       '에스컬레이터-이대역 E/S 2호기 [192070]', '엘리베이터-아현역 1번출구측 엘리베이터 외부2호기',
       '휠체어리프트 내부 2호기(V/L)', '엘리베이터  내부2호기', '휠체어리프트 내부2호기',
       '휠체어리프트 내부5호기', '휠체어리프트 외부1호기', '에스컬레이터 3호',

운행구간

In [70]:
# '운행구간' 2618개, '설치위치' 2959개로 총 3014개 중 각각 빈 값이 있음.
# 왜 빈 값이 존재? -> 1)전 구간을 운행해서 2)설치가 되어있지 않아서
# 만약 가장 가까운 승강기를 찾아주는 프로그램이라면, 빈 값을 어떻게 할 것인지도 논의

elevator['운행구간'].unique()

array(['~', 'B2(대합실)~B1(연결통로)', '섬식 B1(연결통로)~B2(승강장)',
       '섬식 B2(승강장)~B1(연결통로)', '지상~B1', 'B1~지상', 'B2(승강장)~B1(대합실)',
       'B1(대합실)~지상', '지상(시청광장)~B1(대합실)', 'B1(대합실)~지상(시청광장)',
       'B1(대합실)~지상(덕수궁)', '지상~B1(대합실)', 'B1(대합실)~B1(대합실)', 'B2(대합실)~지상',
       'B2(대합실)~B1(승강장)', 'B1(역무실)~지상', 'B2~B3(6호선연결통로)',
       'B3(6호선연결통로)~B2', 'B1(승강장)~B2(연결통로)', 'B2(연결통로)~B1(승강장)',
       '지상~B1(역무실)', 'B1(승강장)~4F', 'B2(연결통로)~BM2-B1(승강장)',
       'B2(환승통로)~B1(승강장)', 'B1(승강장)~BM1-지상', 'B1(승강장)~지상',
       '서울역방향 1-1근접 상행~서울역방향 1-1근접 상행(1)', '내부화장실 근접 상행~내부화장실 근접 상행(2)',
       '서울역방향 1-4 근접 하행~서울역방향 1-4 근접 하행(2)', '1호선(신)~2호선(신)',
       'B3(승강장)~B2(대합실)-B1(대합실)', '지상~연결통로', '연결통로~지상', '연결통로~B1(대합실)',
       'B1(대합실)~연결통로', 'B1(대합실)~BM~지상', 'B1(대합실)~B2(승강장)', '5호선~2호선 환승',
       '2호선~5호선 환승', 'B3(승강장)~B2(대합실)', '지상~F2(대합실)', 'F2(대합실)~지상',
       'F1(승강장)~F2(대합실)', 'F2(대합실)~F3(승강장)', '지상~F2', '지상~F1(대합실)',
       '지상~F2(대합실)-F3(승강장)', 'F1(대합실)~F2(승강장)', 'F3(승강장)~F2(대합실)',
       'B1(상가)~지상', 

설치위치

In [71]:
#nan은 설치가 되어있지 않다는 의미?
#해당 정보를 지도에 표시할 수 있는지 여부?
elevator['설치위치'].unique()

array([nan, '4호선 연결통로 상행', '남영방향 하행 2-3', '시청방향 상행 9-3', '4번출구하행',
       '4번출구상행', '섬식(상) 5-1', '2번출구', '45번출구', '3번출구', '내부 C 계단', '5번출구',
       '1번출구', '상행 3-2', '하행 8-3', '12번출구', '상행 6-4', '하행 4-4', '14번출구',
       '상행(의정부행) 4-4', '하행 6-4', '1213번출구', '상행 8-3', '하행 3-2', '89번출구',
       '6번출구', '상행 10-4', '하행 2-1', '상행 3-4', '하행 8-1', '23번출구',
       '상선 승강장 10-1(내부계단-A)', '하선 승강장 2-1(내부계단-F)', '대합실연결통로(6번출구-1)',
       '용두방향(내선) 승강장 종점 측', '상행(청량리행) 9-4', '하행(서울역행)  2-1', '제기동측 승강장',
       '섬식(상) 8-3', '청량리방향 9-1', '청량리방향 6-2', '시청방향 5-4', '시청방향 2-4',
       '본관건물(상) 6-2', '상행 10-3', '상행 4-8', '하행 7-3', '하행 1-2', '110번출구',
       '1호선 연결통로(구)', '1-2호선(서울역 1호선 쪽) 환승통로',
       '승강장(섬식) 6-2(외선/충정로 측) 5-3(내선/', '9번출구', '8번출구',
       '내선 5-3 대합실(역무실 측)', '외선 6-2 대합실(역무실 측)', '외선(시청행) 8-3',
       '내선(왕십리행) 3-2', '외선 3-2', '내선 8-3', '내선(잠실행) 3-2', '외선(시청행) 3-4',
       '1-1번출구', '외선(신당방향) 8-1', '내선(왕십리방향) 3-4', '외선 7-7', '외선 7-1',
       '내선 2-3', '내선 3-1', '내선 3-4', '6번 출입구', '섬식(

운행상태

In [72]:
elevator['운행상태'].value_counts()

사용가능    3000
공사중       11
보수중        3
Name: 운행상태, dtype: int64

승강기 구분

In [74]:
#장비 구분 : ES(에스컬레이터),EL(엘리베이터)->이게 EV인듯, WL(휠체어 리프트), MW(무빙워크, 에스컬레이터의 일종)
#승강기 구분은 필요한 정보인듯

elevator['승강기 구분'].value_counts()

ES    1936
EV     882
WL     176
MW      20
Name: 승강기 구분, dtype: int64

In [10]:
#모두 문자형 데이터들로 이루어져 있음.

import csv
f = open("/content/drive/MyDrive/2021-1학기/파데분/데이터/서울교통공사 교통약자 이용시설(승강기) 가동현황.csv", encoding='cp949')
data = csv.reader(f)
for row in data :
    print(row)

['역코드', '역명', '승강기명', '운행구간', '설치위치', '운행상태', '승강기 구분']
['150', '서울(1)역', '에스컬레이터 1호기', '~', '', '사용가능', 'ES']
['150', '서울(1)역', '에스컬레이터 1호기', 'B2(대합실)~B1(연결통로)', '4호선 연결통로 상행', '사용가능', 'ES']
['150', '서울(1)역', '에스컬레이터 2호기', '섬식 B1(연결통로)~B2(승강장)', '남영방향 하행 2-3', '사용가능', 'ES']
['150', '서울(1)역', '에스컬레이터 3호기', '섬식 B2(승강장)~B1(연결통로)', '시청방향 상행 9-3', '사용가능', 'ES']
['150', '서울(1)역', '에스컬레이터 4호기', '지상~B1', '4번출구하행', '사용가능', 'ES']
['150', '서울(1)역', '에스컬레이터 5호기', 'B1~지상', '4번출구상행', '사용가능', 'ES']
['150', '서울(1)역', '엘리베이터 내부 1호', 'B2(승강장)~B1(대합실)', '섬식(상) 5-1', '사용가능', 'EV']
['150', '서울(1)역', '엘리베이터 외부 1호기', 'B1(대합실)~지상', '2번출구', '사용가능', 'EV']
['150', '서울(1)역', '엘리베이터 외부 2호기', 'B1(대합실)~지상', '45번출구', '사용가능', 'EV']
['150', '서울(1)역', '엘리베이터 외부3호기', 'B1(대합실)~지상', '3번출구', '사용가능', 'EV']
['150', '서울(1)역', '휠체어리프트 내부 1호기', 'B2(승강장)~B1(대합실)', '내부 C 계단', '사용가능', 'WL']
['151', '시청(1)', '에스컬레이터 1호기', '지상(시청광장)~B1(대합실)', '5번출구', '사용가능', 'ES']
['151', '시청(1)', '에스컬레이터 2호기', 'B1(대합실)~지상(시청광장)', '5번출구', '사용가능', 'E

# 2)역사 공중 화장실 정보

In [25]:
#역 코드가 없음 -> 합친다면 역 명으로 합쳐야 할 것

restroom.head(5)

,구분,화장실명,소재지도로명주소,소재지지번주소,남녀공용화장실여부,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,남성용-어린이용대변기수,남성용-어린이용소변기수,여성용-대변기수,여성용-장애인용대변기수,여성용-어린이용대변기수,관리기관명,전화번호,개방시간,위도,경도,화장실소유구분,화장실설치장소유형,오물처리방식,비상벨 설치유형,화장실입구cctv설치유무,기저귀교환대장소,리모델링년월,데이터기준일자
0,공중화장실,서울역(1),서울특별시 중구 봉래동2가 122 서울역(1호선),서울특별시 중구 봉래동2가,N,4,5,1,0,0,0,9,1,0,서울교통공사,02-6110-1331,05:00~01:00,37.556878,126.972578,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2015,2020-09-28
1,공중화장실,시청역(1),서울특별시 중구 정동 5-5 시청역(1호선),서울특별시 중구 정동 5-5,N,6,5,2,0,2,1,15,2,2,서울교통공사,02-6110-1321,05:00~01:00,37.565682,126.976849,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2013,2020-09-28
2,공중화장실,종각역(1),서울특별시 종로구 종로1가 54 종각지하철역사,서울특별시 종로구 종로1가,N,3,2,2,0,0,1,6,2,1,서울교통공사,02-6110-1311,05:00~01:00,37.570175,126.983226,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2015,2020-09-28
3,공중화장실,종각역(1),서울특별시 종로구 종로1가 54 종각지하철역사,서울특별시 종로구 종로1가,N,5,5,2,0,2,1,10,2,2,서울교통공사,02-6110-1311,05:00~01:00,37.570384,126.992008,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2008이전,2020-09-28
4,공중화장실,종로3가역(1),서울특별시 종로3가 10-5 1호선 종로3가역(1호선),서울특별시 종로3가 10-5,N,5,6,2,0,1,0,12,2,1,서울교통공사,02-6110-1301,05:00~01:00,37.570384,126.992008,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2010,2020-09-28


In [29]:
restroom.tail(5)

,구분,화장실명,소재지도로명주소,소재지지번주소,남녀공용화장실여부,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,남성용-어린이용대변기수,남성용-어린이용소변기수,여성용-대변기수,여성용-장애인용대변기수,여성용-어린이용대변기수,관리기관명,전화번호,개방시간,위도,경도,화장실소유구분,화장실설치장소유형,오물처리방식,비상벨 설치유형,화장실입구cctv설치유무,기저귀교환대장소,리모델링년월,데이터기준일자
300,공중화장실,남한산성입구역(8),경기도 성남시 수정구 단대동 96 남한산성입구역,경기도 성남시 수정구 단대동,N,3,5,1,0,1,0,8,1,1,서울교통공사,02-6311-8221,05:00~01:00,37.451541,127.157632,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2009,2020-09-28
301,공중화장실,단대오거리역(8),경기도 성남시 수정구 신흥동 2467 단대오거리역,경기도 성남시 수정구 신흥동,N,3,4,1,0,1,0,7,1,1,서울교통공사,02-6311-8231,05:00~01:00,37.445163,127.154033,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2009,2020-09-28
302,공중화장실,신흥역(8),경기도 성남시 수정구 신흥동 2467 신흥역,경기도 성남시 수정구 신흥동,N,4,5,1,0,0,0,9,1,0,서울교통공사,02-6311-8241,05:00~01:00,37.440749,127.144161,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2008이전,2020-09-28
303,공중화장실,수진역(8),경기도 성남시 수정구 수진동 2205-1 수진역,경기도 성남시 수정구 수진동,N,3,5,1,0,1,0,10,1,1,서울교통공사,02-6311-8251,05:00~01:00,37.437611,127.138831,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2012,2020-09-28
304,공중화장실,모란역(8),경기도 성남시 수정구 수진동 4808 모란역,경기도 성남시 수정구 수진동,N,5,5,1,0,1,0,10,1,1,서울교통공사,02-6311-8261,05:00~01:00,37.433072,127.127123,서울교통공사,역사 내 화장실,수세식,유,유,"남자화장실,여자화장실",2013,2020-09-28


In [30]:
#'기저귀교환대장소'에 빈 값이 있음 1)기저귀교환대가 없는 화장실일 경우
restroom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   구분             305 non-null    object 
 1   화장실명           305 non-null    object 
 2   소재지도로명주소       305 non-null    object 
 3   소재지지번주소        305 non-null    object 
 4   남녀공용화장실여부      305 non-null    object 
 5   남성용-대변기수       305 non-null    int64  
 6   남성용-소변기수       305 non-null    int64  
 7   남성용-장애인용대변기수   305 non-null    int64  
 8   남성용-장애인용소변기수   305 non-null    int64  
 9   남성용-어린이용대변기수   305 non-null    int64  
 10  남성용-어린이용소변기수   305 non-null    int64  
 11  여성용-대변기수       305 non-null    int64  
 12  여성용-장애인용대변기수   305 non-null    int64  
 13  여성용-어린이용대변기수   305 non-null    int64  
 14  관리기관명          305 non-null    object 
 15  전화번호           305 non-null    object 
 16  개방시간           305 non-null    object 
 17  위도             305 non-null    float64
 18  경도        

구분

In [76]:
# 공중화장실밖에 데이터가 없음 -> 없어도 될 듯
restroom['구분'].value_counts()

공중화장실    305
Name: 구분, dtype: int64

화장실명

In [31]:
restroom_list = restroom['화장실명'].unique()
restroom_list

array(['서울역(1)', '시청역(1)', '종각역(1)', '종로3가역(1)', '종로5가역(1)', '동대문역(1)',
       '동묘앞역(1)', '신설동역(1)', '제기동역(1)', '청량리역(1)', '시청역(2)', '을지로입구역(2)',
       '을지로3가역(2)', '을지로4가역(2)', '동대문역사문화공원역(2)', '신당역(2)', '상왕십리역(2)',
       '왕십리역(2)', '한양대역(2)', '뚝섬역(2)', '성수역(2)', '건대입구역(2)', '구의역(2)',
       '강변역(2)', '잠실나루역(2)', '잠실역(2)', '잠실새내역(2)', '종합운동장역(2)', '삼성역(2)',
       '선릉역(2)', '역삼역(2)', '강남역(2)', '교대역(2)', '서초역(2)', '방배역(2)',
       '사당역(2)', '낙성대역(2)', '서울대입구역(2)', '봉천역(2)', '신림역(2)', '신대방역(2)',
       '구로디지털단지역(2)', '대림역(2)', '신도림역(2)', '문래역(2)', '영등포구청역(2)',
       '당산역(2)', '합정역(2)', '홍대입구역(2)', '신촌역(2)', '이대역(2)', '아현역(2)',
       '충정로역(2)', '신설동역(2)', '용두역(2)', '신답역(2)', '용답역(2)', '도림천역(2)',
       '양천구청역(2)', '신정네거리역(2)', '구파발역(3)', '연신내역(3)', '불광역(3)', '녹번역(3)',
       '홍제역(3)', '무악재역(3)', '독립문역(3)', '경복궁역(3)', '안국역(3)', '종로3가역(3)',
       '을지로3가역(3)', '동대입구역(3)', '약수역(3)', '금호역(3)', '옥수역(3)', '압구정역(3)',
       '신사역(3)', '잠원역(3)', '교대역(3)', '남부터미널역(3)', '양재역(3)', '매봉역(3)',
    

In [36]:
#승강기 역명은 총 277개인 반면 '화장실명'은 총 273개임 -> 차이가 있음

len(restroom_list)

273

In [37]:
#마지막역
print(restroom_list[272])

모란역(8)


1) 승강기 데이터 중 '지축', '강일', '하남시청(덕풍신장)', '하남검산단', '신내'는 역사공중화장실 정보에 없는 역이름이다.

-c.f) '고속터미널', '군자(능동)', '천호(풍납토성)'

-고속터미널역은 3,7,9호선이 있는데 역사공중화장실 정보에는 7호선만 있다.

-군자역은 5,7호선이 있는데 역사공중화장실 정보에는 5호선만 있다.

-천호역은 5,8호선이 있는데 역사 공중화장실 정보에는 5호선만 있다.

=>환승역 기입 등으로 인해 총 수에 차이가 발생. 만약 데이터를 합치게 된다면 고려해야 할 부분임


In [49]:
#환승 정보가 아니라면 중복되는 화장실명은 왜 중복되는 것? 1)화장실이 많아서
restroom['화장실명'].value_counts()

용두역(2)       4
동묘앞역(1)      4
사당역(4)       3
당고개역(4)      2
종로3가역(1)     2
            ..
삼산체육관역(7)    1
충무로역(4)      1
당산역(2)       1
마들역(7)       1
미사역(5)       1
Name: 화장실명, Length: 273, dtype: int64

주소
두 가지 표기가 되어있음

1)소재지도로명주소	2)소재지지번주소

In [77]:
#주소 표기가 필요할까? 싶긴 하다

restroom['소재지도로명주소'].value_counts()

서울특별시 동대문구 용두동 47-1 용두역         4
서울특별시 종로구 숭인동 117 동묘앞역(1호선)     4
서울특별시 동작구 사당동 588-4 사당역(4호선)    3
서울특별시 송파구 오금동 44-2 오금역          3
서울특별시 종로구 묘동 59 종로3가역(3호선)      2
                               ..
서울특별시 영등포구 여의도동 84-4 여의나루역      1
서울특별시 은평구 구산동 1 구산역             1
서울특별시 강동구 명일동 345-12 굽은다리역      1
서울특별시 강동구 암사동 501 암사역           1
서울특별시 영등포구 문래동3가 68-1문래역        1
Name: 소재지도로명주소, Length: 270, dtype: int64

성별 공용 화장실 여부

In [78]:
#모두 N값이니 없어도 무방하지 않을까 싶음.
restroom['남녀공용화장실여부'].value_counts()

N    305
Name: 남녀공용화장실여부, dtype: int64

변기 개수

남성용-대변기수       
남성용-소변기수       
남성용-장애인용대변기수     
남성용-장애인용소변기수     
남성용-어린이용대변기수     
남성용-어린이용소변기수

여성용-대변기수      
여성용-장애인용대변기수  
여성용-어린이용대변기수

1)숫자로 바꾸어서 가장 회전률이 빠른 화장실 구하기? 등으로 이용할 수 있을 것. 

관리기관명

In [3]:
#전부 서울교통공사이므로 지워도 될 것
restroom['관리기관명'].value_counts()

서울교통공사    305
Name: 관리기관명, dtype: int64

전화번호           305 non-null    object 
 16  개방시간           305 non-null    object 
 17  위도             305 non-null    float64
 18  경도             305 non-null    float64
 19  화장실소유구분        305 non-null    object 
 20  화장실설치장소유형      305 non-null    object 
 21  오물처리방식         305 non-null    object 
 22  비상벨 설치유형       305 non-null    object 
 23  화장실입구cctv설치유무  305 non-null    object 
 24  기저귀교환대장소       295 non-null    object 
 25  리모델링년월         305 non-null    object 
 26  데이터기준일자        305 non-null    object 

기타

-전화번호: 필요가 없을 것 같긴 함.

-개방시간

-위도, 경도: 필요 없을 것

-화장실 소유구분: 전부 서울교통공사이므로 없어도 무방

-화장실설치장소유형: 전부 역사 내 화장실로 나옴!

-오물처리방식 : 전부 수세식임. 없어도 될것

-비상벨 설치유형 : 전부 '유'라고 뜬다

-화장실입구cctv설치유무 : 전부 '유'라고 뜬다

-기저귀교환대장소 : 없는 값(NaN)도 있음.

-리모델링년월 : 필요없을듯?

-데이터기준일자 : 데이터 업데이트 일자인듯. 생략해도 무방할것

In [4]:
#전부 서울교통공사이므로 지워도 될 것
restroom['화장실소유구분'].value_counts()

서울교통공사    305
Name: 화장실소유구분, dtype: int64

In [5]:
#모든 데이터가 역사 내 화장실임
restroom['화장실설치장소유형'].value_counts()

역사 내 화장실    305
Name: 화장실설치장소유형, dtype: int64

In [6]:
restroom['오물처리방식'].value_counts()

수세식    305
Name: 오물처리방식, dtype: int64

In [7]:
restroom['비상벨 설치유형'].value_counts()

유    305
Name: 비상벨 설치유형, dtype: int64

In [8]:
restroom['화장실입구cctv설치유무'].value_counts()

유    305
Name: 화장실입구cctv설치유무, dtype: int64

In [9]:
restroom['기저귀교환대장소'].value_counts()

남자화장실,여자화장실    259
여자화장실           35
남자화장실            1
Name: 기저귀교환대장소, dtype: int64

In [11]:
restroom[restroom['기저귀교환대장소'].isnull()]

,구분,화장실명,소재지도로명주소,소재지지번주소,남녀공용화장실여부,남성용-대변기수,남성용-소변기수,남성용-장애인용대변기수,남성용-장애인용소변기수,남성용-어린이용대변기수,남성용-어린이용소변기수,여성용-대변기수,여성용-장애인용대변기수,여성용-어린이용대변기수,관리기관명,전화번호,개방시간,위도,경도,화장실소유구분,화장실설치장소유형,오물처리방식,비상벨 설치유형,화장실입구cctv설치유무,기저귀교환대장소,리모델링년월,데이터기준일자
15,공중화장실,시청역(2),서울특별시 중구 서소문동 90-1 시청역(2호선),서울특별시 중구 서소문동 9,N,5,4,1,0,0,0,10,1,0,서울교통공사,02-6110-2011,05:00~01:00,37.561279,126.969321,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
65,공중화장실,용두역(2),서울특별시 동대문구 용두동 47-1 용두역,서울특별시 동대문구 용두동,N,1,2,2,0,0,1,3,2,1,서울교통공사,02-6110-1361,05:00~01:00,37.554667,126.937500,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
66,공중화장실,용두역(2),서울특별시 동대문구 용두동 47-1 용두역,서울특별시 동대문구 용두동,N,1,2,1,0,0,0,3,1,0,서울교통공사,02-6110-1361,05:00~01:00,37.556813,126.947712,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
67,공중화장실,신답역(2),서울특별시 성동구 용답동 129-2 신답역,서울특별시 성동구 용답동 1,N,2,3,2,0,2,1,3,2,2,서울교통공사,02-6110-1351,05:00~01:00,37.555890,126.955661,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
68,공중화장실,용답역(2),서울특별시 성동구 용답동 127 용답역,서울특별시 성동구 용답동 1,N,2,3,2,0,0,0,4,2,0,서울교통공사,02-6110-1341,05:00~01:00,37.559493,126.963818,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
81,공중화장실,종로3가역(3),서울특별시 종로구 묘동 59 종로3가역(3호선),서울특별시 종로구 묘동 59,N,5,5,2,0,1,1,15,2,2,서울교통공사,02-6110-3291,05:00~01:00,37.559227,127.005275,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2010,2020-09-28
105,공중화장실,경찰병원역(3),서울특별시 송파구 가락동 10-15 경찰병원역,서울특별시 송파구 가락동 1,N,3,4,2,0,1,1,11,2,1,서울교통공사,02-6110-3512,05:00~01:00,37.503246,127.126342,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2010,2020-09-28
140,공중화장실,김포공항역(5),서울특별시 강서구 방화동 886 김포공항역,서울특별시 강서구 방화동 8,N,1,2,1,0,0,0,2,1,0,서울교통공사,02-6311-5121,05:00~01:00,37.561829,126.799739,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2008이전,2020-09-28
285,공중화장실,삼산체육관역(7),인천광역시 부평구 삼산동 466 삼산체육관역,인천광역시 부평구 삼산동 4,N,5,5,1,0,0,0,6,1,0,서울교통공사,02-6311-7571,05:00~01:00,37.506483,126.739835,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2012,2020-09-28
286,공중화장실,굴포천역(7),인천광역시 부평구 삼산동 466 굴포천역,인천광역시 부평구 삼산동 4,N,7,6,1,0,1,0,13,1,1,서울교통공사,02-6311-7581,05:00~01:00,37.508231,126.723624,서울교통공사,역사 내 화장실,수세식,유,유,NaN,2012,2020-09-28


In [12]:
restroom['리모델링년월'].value_counts()

2008이전     89
2009       45
2010       36
2012       28
2015       17
2019       15
2018       14
2013       14
2017       10
2014       10
2016        8
2011        7
2008 이전     5
2008        4
2020        2
2006        1
Name: 리모델링년월, dtype: int64

In [13]:
restroom['데이터기준일자'].value_counts()

2020-09-28    305
Name: 데이터기준일자, dtype: int64

In [50]:
#변기의 수, 날짜, 년도 등이 문자열 형식 -> 사용한다면 숫자로 바꿔주어야 할 것.

import csv
f = open("/content/drive/MyDrive/2021-1학기/파데분/데이터/서울교통공사_역사공중화장실정보_20200928.csv", encoding="cp949")
data = csv.reader(f)
for row in data :
    print(row)

['구분', '화장실명', '소재지도로명주소', '소재지지번주소', '남녀공용화장실여부', '남성용-대변기수', '남성용-소변기수', '남성용-장애인용대변기수', '남성용-장애인용소변기수', '남성용-어린이용대변기수', '남성용-어린이용소변기수', '여성용-대변기수', '여성용-장애인용대변기수', '여성용-어린이용대변기수', '관리기관명', '전화번호', '개방시간', '위도', '경도', '화장실소유구분', '화장실설치장소유형', '오물처리방식', '비상벨 설치유형', '화장실입구cctv설치유무', '기저귀교환대장소', '리모델링년월', '데이터기준일자']
['공중화장실', '서울역(1)', '서울특별시 중구 봉래동2가 122 서울역(1호선)', '서울특별시 중구 봉래동2가 ', 'N', '4', '5', '1', '0', '0', '0', '9', '1', '0', '서울교통공사', '02-6110-1331', '05:00~01:00', '37.556878', '126.972578', '서울교통공사', '역사 내 화장실', '수세식', '유', '유', '남자화장실,여자화장실', '2015', '2020-09-28']
['공중화장실', '시청역(1)', '서울특별시 중구 정동 5-5 시청역(1호선)', '서울특별시 중구 정동 5-5', 'N', '6', '5', '2', '0', '2', '1', '15', '2', '2', '서울교통공사', '02-6110-1321', '05:00~01:00', '37.565682', '126.976849', '서울교통공사', '역사 내 화장실', '수세식', '유', '유', '남자화장실,여자화장실', '2013', '2020-09-28']
['공중화장실', '종각역(1)', '서울특별시 종로구 종로1가 54 종각지하철역사', '서울특별시 종로구 종로1가 ', 'N', '3', '2', '2', '0', '0', '1', '6', '2', '1', '서울교통공사', '02-6110-1311', '05:00~01:00',

# 3)편의시설

In [14]:
amenities.head(5)

,역번호,역명,외부역번호,호선,문화공간여부,휠체어리프트여부,만남의장소여부,환승주차장여부,자전거보관소여부,엘리베이터여부,기차예매역여부,무인민원발급기여부,환전키오스크여부,수유실여부
0,150,서울역,133,01호선,N,Y,N,N,N,Y,N,N,N,N
1,151,시청,132,01호선,N,N,N,N,N,Y,N,N,N,N
2,152,종각,131,01호선,N,N,N,N,Y,Y,N,Y,N,N
3,153,종로3가,130,01호선,N,N,N,N,N,Y,N,N,N,Y
4,154,종로5가,129,01호선,N,N,N,N,N,Y,N,N,N,N


In [15]:
amenities.tail(5)

,역번호,역명,외부역번호,호선,문화공간여부,휠체어리프트여부,만남의장소여부,환승주차장여부,자전거보관소여부,엘리베이터여부,기차예매역여부,무인민원발급기여부,환전키오스크여부,수유실여부
289,4134,송파나루,934,09호선,N,N,N,N,N,Y,N,Y,N,Y
290,4135,한성백제,935,09호선,N,N,N,N,N,Y,N,N,N,Y
291,4136,올림픽공원,936,09호선,N,N,N,N,N,Y,N,N,N,Y
292,4137,둔촌오륜,937,09호선,N,N,N,N,N,Y,N,N,N,Y
293,4138,중앙보훈병원,938,09호선,N,N,N,N,N,Y,N,N,N,Y


In [16]:
#null값은 없음
amenities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   역번호        294 non-null    int64 
 1   역명         294 non-null    object
 2   외부역번호      294 non-null    object
 3   호선         294 non-null    object
 4   문화공간여부     294 non-null    object
 5   휠체어리프트여부   294 non-null    object
 6   만남의장소여부    294 non-null    object
 7   환승주차장여부    294 non-null    object
 8   자전거보관소여부   294 non-null    object
 9   엘리베이터여부    294 non-null    object
 10  기차예매역여부    294 non-null    object
 11  무인민원발급기여부  294 non-null    object
 12  환전키오스크여부   294 non-null    object
 13  수유실여부      294 non-null    object
dtypes: int64(1), object(13)
memory usage: 32.3+ KB


역번호

In [17]:
amenities['역번호'].unique()

array([ 150,  151,  152,  153,  154,  155,  156,  157,  158,  159,  200,
        201,  202,  203,  204,  205,  206,  207,  208,  209,  210,  211,
        212,  213,  214,  215,  216,  217,  218,  219,  220,  221,  222,
        223,  224,  225,  226,  227,  228,  229,  230,  231,  232,  233,
        234,  235,  236,  237,  238,  239,  240,  241,  242,  243,  244,
        245,  246,  247,  248,  249,  250,  309,  310,  311,  312,  313,
        314,  315,  316,  317,  318,  319,  320,  321,  322,  323,  324,
        325,  326,  327,  328,  329,  330,  331,  332,  333,  334,  335,
        336,  337,  338,  339,  340,  341,  342,  409,  410,  411,  412,
        413,  414,  415,  416,  417,  418,  419,  420,  421,  422,  423,
        424,  425,  426,  427,  428,  429,  430,  431,  432,  433,  434,
       2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521,
       2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532,
       2533, 2534, 2535, 2536, 2537, 2538, 2539, 25

In [18]:
#편의시설의 역번호는 294개
len(amenities['역번호'].unique())

294

역명

1호선~9호선인것 같음

In [19]:
#앞의 두 데이터와는 다르게 호선이 표시되어 있지 않은 역이름
amenities['역명'].unique()

array(['서울역', '시청', '종각', '종로3가', '종로5가', '동대문', '신설동', '제기동', '청량리',
       '동묘앞', '까치산', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리',
       '왕십리', '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
       '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
       '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
       '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '도림천',
       '양천구청', '신정네거리', '용두', '지축', '구파발', '연신내', '불광', '녹번', '홍제', '무악재',
       '독립문', '경복궁', '안국', '충무로', '동대입구', '약수', '금호', '옥수', '압구정', '신사',
       '잠원', '고속터미널', '남부터미널', '양재', '매봉', '도곡', '대치', '학여울', '대청', '일원',
       '수서', '가락시장', '경찰병원', '오금', '당고개', '상계', '노원', '창동', '쌍문', '수유',
       '미아', '미아사거리', '길음', '성신여대입구', '한성대입구', '혜화', '명동', '회현', '숙대입구',
       '삼각지', '신용산', '이촌', '동작', '총신대입구', '남태령', '방화', '개화산', '김포공항',
       '송정', '마곡', '발산', '우장산', '화곡', '신정', '목동', '오목교', '양평', '영등포시장',
       '신길', '여의도', '여의나루', '마포', '공덕', '애오개', '서대문', '

In [20]:
len(amenities['역명'].unique())

255

In [21]:
#역번호는 294개, 역명은 255개 -> 39개만큼의 차이가 있음.
amenities['역명'].value_counts()

종로3가         3
동대문역사문화공원    3
동묘앞          2
을지로3가        2
천호           2
            ..
춘의           1
이촌           1
상동           1
홍제           1
남태령          1
Name: 역명, Length: 255, dtype: int64

역번호와 역명의 차이: 환승역 때문일것

: 가락시장, 건대입구락시장, 건대입구, 교대, 군자, 까치산, 노원, 대림..

예외) 환승역이면서 중복되지 않은 역

: 가산디지털단지역, 

외부 역번호
: 편의시설에는 외부 역번호정보도 있음. 그러나 위의 두 데이터와 딱히 연관관계는 없으므로 사용하지 않아도 괜찮을 듯.

호선

기타 편의시설 

: 단지 존재하는가의 여부.. 흠

-문화공간여부

-휠체어리프트여부

-만남의장소여부

-환승주차장여부

-자전거보관소여부

-엘리베이터여부

-기차예매역여부

-무인민원발급기여부

-환전키오스크여부

-수유실여부


# 데이터 방향성